#### Introduction


In this tutorial, we will refine a transformation model for *Named Entity Recognition* based on the BETO model (a Spanish adaptation of the BERT model). 
This tutorial aims at producing a tranformers model via the spacy and sklearn library by proceeding to the cross validation technique.

**Warning** : This processing chain works only via the notebook interface (when using the GPU colab). It is possible to use the native *subsystem* library to execute bash commands.

##### **Configuration tu run colab in local environnement**
- Step 1: Install Jupyter

Install Jupyter on your local machine.
- Step 2: Install the jupyter_http_over_ws extension and activate it (one time only)

Created by the Colaboratory team, the jupyter_http_over_ws extension is available on GitHub.

`pip install jupyter_http_over_ws jupyter serverextension enable --py jupyter_http_over_ws`

- Step 3: Start the server and perform authentication

Even if the new notebook servers are started normally, you must set a flag to clearly approve WebSocket connections from the Colaboratory interface.

`jupyter notebook \
  --NotebookApp.allow_origin='https://colab.research.google.com' \
  --port=8888 \
  --NotebookApp.port_retries=0`
    

After starting, the server will display a message containing the initial backend URL used for authentication. Copy this URL, as you will need it in the next step.
- Step 4: Connect to the local runtime environment

In Colaboratory, click on the "Connect" button, and then select "Connect to a local runtime environment". In the dialog box that appears, enter the URL you copied in the previous step and click the "Connect" button. Now you should be connected to your local execution environment. 

##### Flow of the notebook

The notebook will be divided into seperate sections to provide a organized walk through for the process used.

1. [Checking system and requirements](#section00)
2. [Configuration system](#section01)
3. [Finetuning](#section02) \
3-1. [Preprocessing](#section021) \
3-2. [Training](#section022)
4. [Global evaluation](#section03)

<a name="section00"></a>
####Checking

In [ ]:
#Check gpu activity
!nvidia-smi

Sun Jul 24 01:42:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.57       Driver Version: 516.59       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:07:00.0  On |                  N/A |
|  0%   45C    P8    29W / 370W |    648MiB / 10240MiB |     37%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##### install dependencies

In [ ]:
!pip install -U pandas
!pip install -U scikit-learn
#To use your GPU in local you need to install the CUPY version CUDA of your system
#https://github.com/explosion/spacy-transformers
!pip install -U spacy[transformers,cuda117]

Requirement already up-to-date: pandas in ./.env/lib/python3.8/site-packages (1.4.3)
Requirement already up-to-date: scikit-learn in ./.env/lib/python3.8/site-packages (1.1.1)
Requirement already up-to-date: spacy[cuda117,transformers] in ./.env/lib/python3.8/site-packages (3.4.0)
  Using cached cupy_cuda117-10.6.0-cp38-cp38-manylinux1_x86_64.whl (83.5 MB)
  Using cached fastrlock-0.8-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (49 kB)


In [ ]:
#If your use a RTX3000, you need to install preview package of pytorch (Here works on = or > CUDA 116)
!pip uninstall torch torchvision torchaudio
!pip install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cu116
     |████████████████████████████████| 23.8 MB 23.6 MB/s 
     |████████████████████████████████| 3.9 MB 754 kB/s 
  Using cached Pillow-9.2.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
ERROR: torchvision 0.14.0.dev20220723+cu116 has requirement torch==1.13.0.dev20220723+cu116, but you'll have torch 1.12.0 which is incompatible.
ERROR: torchaudio 0.13.0.dev20220723+cu116 has requirement torch==1.13.0.dev20220723, but you'll have torch 1.12.0 which is incompatible.


<a name="section01"></a>
####Configuration

In [ ]:
#for dev and train data
import os
#specific train
if not os.path.isdir('./nlp'):
  !mkdir ./nlp
#data
if not os.path.isdir('./nlp/spacyNER_data'):
  !mkdir ./nlp/spacyNER_data
#model 
if not os.path.isdir('./nlp/spacyNER_model'):
  !mkdir ./nlp/spacyNER_model
#cross_validation
if not os.path.isdir('./nlp/spacyNER_model/cross_valid'):
 !mkdir ./nlp/spacyNER_model/cross_valid

#####Checking GPU works

In [ ]:
import cupy as cp

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import spacy
spacy.require_gpu()

True

<a id='section02'></a>
### Finetuning

<a id='section021'></a>
####Preprocessing

In [ ]:
from pandas import DataFrame

def split_dataset(dataframe : DataFrame, train_ratio: float):
  """
  function to split dataframe with ration as you want
  :dataframe: dataframe
  :train_ratio: float, ratio of split training
  :return: None
  """
  from sklearn.model_selection import train_test_split

  assert(train_ratio < 1), "the number must be value between 0 and 1"

  train_df, test_df = train_test_split(dataframe, test_size= 1 - train_ratio)
  
  return train_df, test_df

In [ ]:
from pandas import DataFrame

def convert_binary(name: str, data: DataFrame):
  """
  function to convert dataset (Dataframe) in binary spacy format
  :name: str, name of binary file outpout
  :data: dataframe
  :return: count of tokens and entities in file
  """
  from spacy.tokens import DocBin
  
  #Generate tokenization
  nlp = spacy.blank("es")
  # the DocBin will store the example documents
  db = DocBin(attrs=["ENT_IOB", "ENT_TYPE"])

  #counting variable
  n_token = 0
  n_entities = 0

  for index, row in data.iterrows():
    doc = nlp(row["text"])
    n_token += len(doc)
    n_entities += len(row["label"])
    ents = []
    for ent in row["label"]:
      start, end, label = tuple(ent)
      span = doc.char_span(start, end, label=label)
      if span is not None:
        ents.append(span)
      else:
        n_entities -= 1
    try:
      doc.ents = ents
    except TypeError:
      print(ents)
      pass
    db.add(doc)
    db.to_disk(f"./nlp/spacyNER_data/{name}.spacy")
  return n_token, n_entities

In [ ]:
def export_json(data: dict):
  """
  Export json file with metadata's models
  
  :data: dictionary of metadata
  """
  import json

  with open("./nlp/spacyNER_model/cross_valid/metadata.json", "w", encoding="utf-8") as json_files:
    json.dump(data, json_files, ensure_ascii=False, indent=3)

<a name="section022"></a>
#### Training

In [ ]:
# Configuration file
config = '/content/drive/MyDrive/nlp/base_config_NoTransformers.cfg' #@param {type:"string"}

In [ ]:
from sklearn.model_selection import KFold
import pandas as pd

#import dataset
df = pd.read_json("./nlp/spacyNER_data/dataset_araucania.jsonl", lines = True, encoding="utf-8")

#structure dataset

dataset = split_dataset(df, 0.90)

X = dataset[0]
test_dataset = convert_binary("test", dataset[1])
print("test: " + str(test_dataset[0]) + " tokens, " + str(test_dataset[1]) + " entities")

#K-Fold configuration
kf = KFold(n_splits=6, shuffle = True, random_state = 2)

#variables
n = 0
results = {}

#loop on k
for train_index , test_index in kf.split(X):
  
  #enumeration
  n += 1
  print("<----- run k " + str(n) + "----->")

  #path
  os.mkdir(f"./nlp/spacyNER_model/cross_valid/fold_{str(n)}")
  path_ouput = f"./nlp/spacyNER_model/cross_valid/fold_{str(n)}"
  best_model = f"./nlp/spacyNER_model/cross_valid/fold_{str(n)}/model-best"
  json_output = f"./nlp/spacyNER_model/cross_valid/fold_{str(n)}/scores.json"

  #convert in binary format
  X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
  train_binary = convert_binary("train",X_train)
  dev_binary = convert_binary("dev",X_test)

  #metadata
  results[n] ={
                    "test_data": [test_dataset[0],test_dataset[1]],
                    "train_data" : [train_binary[0],train_binary[1]],
                    "dev_data" : [dev_binary[0],dev_binary[1]]
                }

  #init config
  !python -m spacy init fill-config $config config.cfg

  #training
  !python -m spacy train config.cfg --paths.train ./nlp/spacyNER_data/train.spacy --paths.dev ./nlp/spacyNER_data/dev.spacy -g 0 --output $path_ouput
  
  #evaluation
  !python -m spacy evaluate $best_model ./nlp/spacyNER_data/test.spacy --output $json_output --gold-preproc --gpu-id 0

#export json metadata
export_json(results)

test: 1984 tokens, 174 entities
<----- run k 1----->
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
ℹ Saving to output directory: nlp/spacyNER_model/cross_valid/fold_1
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-07-24 01:43:15,615] [INFO] Set up nlp object from config
[2022-07-24 01:43:15,622] [INFO] Pipeline: ['transformer', 'ner']
[2022-07-24 01:43:15,625] [INFO] Created vocabulary
[2022-07-24 01:43:15,625] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transfor

<a name="section03"></a>
### Global Evaluation

#####Choose model

Put the path of your select model

In [ ]:
path_model = './nlp/spacyNER_model/cross_valid/fold_6/model-best' #@param {type:"string"}

####Metrics

#### Visualisation Model

In [ ]:
#init GPU spacy
import spacy
gpu = spacy.prefer_gpu()
print('GPU:', gpu)

GPU: True


##### Choose text to analyze

`text_clean` corresponds to a corrected and modernized transcription to the contemporary language. `text_brut` corresponds to the spellchecker/postprocess output of XML-ALTO. Finally, `text_htr` is an HTR transcription without filter or correction.
The purpose of these different transcriptions is to visually observe the efficiency of the NER model

In [ ]:
text_clean = """
Comandancia de Armas
Arauco Diciembre 14 de 1859
En este momento recibí la nota de Usted fecha de ayer.
Inmediatamente hice propio a Los Ángeles, participando al Señor Intendente, el éxito que ha tenido.
Tendré mucho cuidado en sujetar todos los animales que arreen ilegalmente.
Todavía no he recibido los 80 animales que le mandé, para hacer la correspondiente devolución a sus dueños.
Aquí no hay novedad. El conductor don Salvador Hermosilla lleva 25 lanzas, con éstas son 124.
Dios guarde a Usted,
José del Carmen Díaz
Al Señor Comandante en Jefe de la División Pacificadora de Arauco

--------------------

Gobierno Interino de
Arauco Enero 2 de 1860
Por el Gobernador del Departamento de Lautaro se me comunica lo que sigue:
Santa Juana, Enero 2 de 1860
Por la Intendencia de mi provincia en nota oficial fecha 29 del mes próximo pasado Nº 472, se me ordena poner a disposición de Usted al reo Juan [Hermosilla], titulado Sargento Mayor de la montonera de Patricio Silva; para que allí sea juzgado, y en su consecuencia se lo remito bajo segura custodia, y Usted se servirá acusar recibo. Dios guarde a Usted, Pascual Ruiz
Yo lo transcribo a Usted para su conocimiento.
Mientras Usted se sirva determinar de dicho reo, he dispuesto mandarlo a bordo del Vapor “Maipú”, para la mayor seguridad.
Dios guarde a Usted,
José del Carmen Díaz
"""

text_brut = """"""

text_htr = """
Comand^oo de armas
En este promente recibi la nota de Ue pha. de Aytt.
Inmediatamente hire prepio a los Anpeles, participando al Sõr Intend^te, el endito que ha tenido.
Tendré mucho cuidado en sufetar todos los animales que Vanien ilegalmente.
Jodabia no he recibo do los 80 animales que le mande, para hacer la corespondiente débolucion asus dueños
Aqui no hai novedad.
El conductor D^n Salvador Elmnosilla lleba 25 lanzar, con estas son 124.
Naueo Obre. 14 de 1859.
Dios gue. aUd.
Jdel C. Dize
Al Señor Coronel Comand^
U Jefe dela Divicion pacificadora de Arauco.

--------------------

E^te inteime de
or el Gobemador del dep^t de Sauteno se me comunica lo que sigue.
Santa Juana Conero 2 de 1860.
Por la Intend^a demi provincia en nota oficial tha 29 del mes ep^a No N72, seme ordena poner a disporicion de Ud al reo Juan llmorcilla, tetulado Sarjento mayor dela montonera de Patricio Silva; para que allí sea juegado; i en su concecuencio selo remito bajo segura custodia; i Ud.
se servirá acuzarme recibo. Dios que a Us Parcual Ruir.
Tolo lo trarcribo a Ul para su conocimiento.
Mientras Ud se sirva determinas de dicho reo, he dispueto mandarlo abordo del vapor Maipú, para la mayor seguridad.
Arauco Lonezo 2 de 1860
No 3
35
1
Al Sõr Comand^o en fefe dela Piarcion deoperaciones de Arauco.
Dios gu~e a Ud
Jel C. Diaz
"""

In [ ]:
text = text_clean #@param ["text_clean", "text_brut", "text_htr"] {type:"raw"}

##### Process
If you want to use a non-transformers model, you need to install spacy classical version

In [ ]:
#Processing tokenization
nlp = spacy.load(path_model)
doc_clean = nlp(text)

In [ ]:
from spacy import displacy
from pathlib import Path

#option visualizers ent
colors = {
    "MISC": "#808D8E",
    "LOC": "#766C7F",
    "PERS": "#947EB0",
    "DATE": "#A3A5C3",
    "ORG": "#A9D2D5"
    }
options= {"ents": ["MISC", "LOC", "PERS", "DATE", "ORG"], "colors": colors}

#render
html = displacy.render(doc_clean, style="ent", jupyter=True, options=options, page=True)